## Paso 5: Técnica de los K Vecinos más Cercanos

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OrdinalEncoder
from itertools import combinations
import time

def preprocess_data(df):
    # Imputar valores ausentes con la moda de cada columna
    df.fillna(df.mode().iloc[0], inplace=True)
    
    # Gestión de datos categóricos de tipo ordinal: se tratarán a través de la técnica de Ordinal Encoder
    ordinal_encoder = OrdinalEncoder()
    df[['loan_grade']] = ordinal_encoder.fit_transform(df[['loan_grade']])

    # Se utilizarán el 75% de los datos para construir el modelo y el 25% restante para validar el modelo construido
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['loan_status']), df['loan_status'], test_size=0.25, random_state=42)
    
    return X_train, X_test, y_train, y_test, ordinal_encoder

def generate_knn_dataframe(df, cols, k, ordinal_encoder):
    X = df[list(cols)]
    y = df['loan_status']

    # Normalizar o estandarizar los datos si es necesario (en este caso, no se necesita)
    # No se realiza ninguna transformación en los datos

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    start_time_model_generation = time.time()
    # Se crea y entrena el modelo de clasificación de K Vecinos más Cercanos
    knn_classifier = KNeighborsClassifier(n_neighbors=k)
    model = knn_classifier.fit(X_train, y_train)
    model_generation_time = time.time() - start_time_model_generation

    start_time_model_testing = time.time()
    # Se predicen las salidas de datos de prueba
    y_test_pred = model.predict(X_test)

    # Se calculan las métricas para evaluar el modelo de clasificación
    mc = confusion_matrix(y_test, y_test_pred)
    e = accuracy_score(y_test, y_test_pred)
    p = precision_score(y_test, y_test_pred)
    s = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    model_testing_time = time.time() - start_time_model_testing

    return mc, e, p, s, f1, model_generation_time, model_testing_time

if __name__ == "__main__":
    # Se cargan los datos en un DataFrame
    df = pd.read_csv("datos/credit_risk_dataset.csv")

    # Se generan todas las combinaciones posibles de columnas
    todas_columnas = ['person_age', 'person_income', 'cb_person_cred_hist_length', 'loan_grade']
    combinaciones_columnas = []
    for r in range(1, len(todas_columnas) + 1):
        combinaciones_columnas.extend(combinations(todas_columnas, r))

    # Se definen los valores de k para los vecinos más cercanos
    valores_k = [1, 2, 3]

    # Preprocesamiento de datos
    start_time_preprocessing = time.time()
    X_train, X_test, y_train, y_test, ordinal_encoder = preprocess_data(df)
    preprocessing_time = time.time() - start_time_preprocessing
    print("Tiempo de preprocesamiento:", preprocessing_time)

    # Se itera sobre cada valor de k
    for k in valores_k:
        print(f"Resultados para K={k}:")
        
        # Se itera sobre cada combinación de columnas
        for cols in combinaciones_columnas:
            mc, e, p, s, f1, model_generation_time, model_testing_time = generate_knn_dataframe(df, cols, k, ordinal_encoder)

            # Mostrar resultados
            print(f"Columnas: {cols}")
            print('Matriz de Confusión (MC):\n', mc)
            print('Exactitud (E):', e)
            print('Precisión (P):', p)
            print('Sensibilidad (S):', s)
            print('F1 Score (F1):', f1)
            print('Tiempo de generación del modelo:', model_generation_time)
            print('Tiempo de prueba del modelo:', model_testing_time)
            print('-' * 50)


Tiempo de preprocesamiento: 0.02602362632751465
Resultados para K=1:
Columnas: ('person_age',)
Matriz de Confusión (MC):
 [[4952 1379]
 [1458  357]]
Exactitud (E): 0.6517309108765038
Precisión (P): 0.2056451612903226
Sensibilidad (S): 0.19669421487603306
F1 Score (F1): 0.20107012109265
Tiempo de generación del modelo: 0.006216764450073242
Tiempo de prueba del modelo: 0.4571671485900879
--------------------------------------------------
Columnas: ('person_income',)
Matriz de Confusión (MC):
 [[4830 1501]
 [ 905  910]]
Exactitud (E): 0.7046403142646698
Precisión (P): 0.37743674823724593
Sensibilidad (S): 0.5013774104683195
F1 Score (F1): 0.4306672976810222
Tiempo de generación del modelo: 0.006993770599365234
Tiempo de prueba del modelo: 0.33016037940979004
--------------------------------------------------
Columnas: ('cb_person_cred_hist_length',)
Matriz de Confusión (MC):
 [[4694 1637]
 [1329  486]]
Exactitud (E): 0.6358949177510435
Precisión (P): 0.2289213377296279
Sensibilidad (S): 0